# Micro- (and hopefully soon geo-) Founded Occupational Mobility Network

*Setup from @rmaria del rio-chanona et al. 2021*
*Code: @ebbamark


In [1]:
# Import packages
import numpy as np
import pandas as pd
import random as random
import matplotlib.pyplot as plt
import math as math
from scipy.interpolate import splrep, BSpline
import seaborn as sns
from IPython import display
from matplotlib.backends.backend_pdf import PdfPages
rng = np.random.default_rng()

path = "~/Documents/Documents - Nuff-Malham/GitHub/transition_abm/"

## Setup

### Agents and Classes

One function and three classes are defined:
- (Function) Utility/decision-making function used by workers when deciding which vacancies to apply to.
- (Class) Worker: individual worker has state-specific attributes (whether or not employed, current or latest held occupation, time employed or unemployed, current or latest wage held, whether or not they have been hired in a particular time step) and character-specific attributes (occupational history, risk aversion score (not yet implemented) and an impatience score (not yet used)). Worker has one function which is to search and apply for a vacancy.
- (Class) Occupation has an id, list of workers currently employed in that occupation, list of neighboring occupations based on transition adjacency matrix (imperfect solution), current and target demand for labour, list of applicants to open vacancies, and wage). Occupation has two internal functions (1) to separate workers and (2) to update all workers in an occupation after each time step.
- (Class) Vacancy has an occupational id, list of applicants (duplicated above in occupation class...to fix), and a wage (duplicated above in occupation class...to fix). Vacancy has one internal function to hire an applicant.

In [2]:
## Defining functions
# Ranking utility/decision-making function
def util(w_current, w_offered, skill_sim):
    # No longer scale by impatience
    # return 1/(1+(math.exp(-impatience_factor*((w_offered-(w_current*(1-skill_sim)))/10000))))
    return 1/(1+(math.exp(-((w_offered - w_current)/10000))))

# Simple quadratic for now in which a worker increase search effort for a period of 6 time steps (ie. months) 
# unemployed after which a worker begins to become discouraged. 
# This follows definition from the US BLS and Pew Research Centre
def search_effort(t_unemp):
    return round(20/((t_unemp-6)**2 + 1)) + 1

## Defining classes
# Potentially redundant use of IDs in the below classes...to check
class worker:
    def __init__(wrkr, occupation_id, employed, longterm_unemp, time_employed,
                 time_unemployed, wage, hired, list_of_occs, risk_av_score):
        # State-specific attributes:
        # Occupation id
        wrkr.occupation_id = occupation_id
        # Binary variable for whether employed or not
        wrkr.employed = employed
        # Binary variable for whether long-term unemployed
        wrkr.longterm_unemp = longterm_unemp
        # Number of time steps employed
        wrkr.time_employed = time_employed
        # Number of time steps unemployed (perhaps redundant with above)
        # Used as criteria for impatience
        wrkr.time_unemployed = time_unemployed
        # Worker wage
        # Could be used as additional criteria for impatience...
        wrkr.wage = wage
        # Whether the worker has been hired in this time step - reset to zero at the end of every time step
        # Used as protective attribute in hiring process (ie. cannot be hired twice)
        wrkr.hired = hired
        
        # Character-specific attributes:
        # Employment history, list of occupations previously held
        # NOT YET USED
        wrkr.emp_history = list_of_occs
        # Identity score - to be defined...
        # wrkr.identity = identity_score
        # Risk aversion: Stefi suggested to use number of 
        # occupations previously held as proxy ie. len(emp_history)
        # Currently takes a value 0-9 indicating at which index of utility ranked vacancies to start sampling/slicing
        wrkr.risk_aversion = risk_av_score
    
    def search_and_apply(wrkr, net, vac_list, beh):
        # A sample of relevant vacancies are found that are in neighboring occupations
        # Will need to add a qualifier in case sample is greater than available relevant vacancies
        # ^^ have added qualifier...bad form to reassign list?
        rel_vacs = [vac for vac in vac_list if net[wrkr.occupation_id].list_of_neigh_bool[vac.occupation_id]]
        if beh:
            rel_vacs = random.sample(rel_vacs, min(len(rel_vacs), 30))
            # Sort found relevant vacancies by utility-function defined above and apply to amount dictated by impatience
            for v in sorted(rel_vacs, key = lambda v: util(wrkr.wage, v.wage,net[wrkr.occupation_id].list_of_neigh_weights[v.occupation_id]), reverse = True)[slice(wrkr.risk_aversion, wrkr.risk_aversion + search_effort(wrkr.time_unemployed))]:
                # Introduce randomness here...binomial?
                v.applicants.append(wrkr)
        else:
            rel_vacs = random.sample(rel_vacs, min(len(rel_vacs), search_effort(wrkr.time_unemployed)))
            for r in rel_vacs:
                r.applicants.append(wrkr)
            
class occupation:
    def __init__(occ, occupation_id, list_of_workers, list_of_neigh_bool, 
                 list_of_neigh_weights, current_demand, 
                 target_demand, applicants, wage):
        occ.occupation_id = occupation_id
        occ.list_of_workers = list_of_workers
        occ.list_of_neigh_bool = list_of_neigh_bool
        occ.list_of_neigh_weights = list_of_neigh_weights
        occ.current_demand = current_demand
        occ.target_demand = target_demand
        occ.applicants = applicants
        occ.wage = wage
    
    def separate_workers(occ, delta_u):
        if(len(occ.list_of_workers) != 0):
            sep_prob = delta_u + gamma * max(0, occ.current_demand - occ.target_demand)/(sum(wrkr.employed for wrkr in occ.list_of_workers) + 1)
            emp = [el for el in occ.list_of_workers if el.employed]
            sep_counter = 0
            print("sep_prob: ",sep_prob)
            for w in random.sample(emp, np.random.binomial(len(emp), sep_prob)):
                w.employed = False
                w.longterm_unemp = False
                w.time_employed = 0
                w.time_unemployed = 0
                sep_counter += 1
    
    def update_workers(occ):
        for w in occ.list_of_workers:
            # Must update hired attribute of workers
            w.hired = False
            if w.employed:
                w.time_employed += 1
            if not(w.employed):
                w.time_unemployed += 1
                w.longterm_unemp = True if w.time_unemployed >= 7 else False
                #don’t w.search_effort = search_effort(w.time_unemployed)
                
        
class vac:
    def __init__(v, occupation_id, applicants, wage):
        v.occupation_id = occupation_id
        v.applicants = applicants
        v.wage = wage
    def hire(v, net):
        a = random.choice([app for app in v.applicants if not(app.hired)])
        assert(not(a.employed))
        assert(not(a.hired))
        net[v.occupation_id].list_of_workers.append(net[a.occupation_id].list_of_workers.pop(net[a.occupation_id].list_of_workers.index(a)))
        a.occupation_id = v.occupation_id
        a.employed = True
        a.longterm_unemp = False
        a.time_employed = 0
        a.time_unemployed = 0
        a.wage = v.wage
        a.emp_history.append(v.occupation_id)
        a.hired = True
        # Reset?
        # wrkr.risk_aversion = risk_av_score
        # Reset?
        # wrkr.impatience = impatience_score
        v.applicants.clear()

        
def bus_cycle_demand(d_0, time, amp, period):
    """function that target demand of time t of sigmoid shock with parameters
    Args:
        d_0: vector of initial demand of occupation
        d_final: (ignored)
        amplitude: amplitude of business cycle
        period: period for full business cycle
    Returns
        d_dagger(Array{Float64,2}): demand of occupation at time t
    """
#     if t < t_shock:
#         return d_0
#    else:
        # start cycle when shock starts
        #t0 = t + t_shock
    d_target =  d_0 * (1 - amp * np.sin((2*np.pi / period) * time))
    return d_target

#x1 = range(10)
#plot(range(10), bus_cycle_demand(1000, range(10), 0.9, 15))
#y1 = bus_cycle_demand(1000, range(10), 0.9, 15)
# data = {'a': np.arange(10)}
# data['b'] = bus_cycle_demand(1000, data['a'], 0.9, 15)
# data['c'] = bus_cycle_demand(1000, data['a'], 0.5, 15)


# plt.scatter('a', 'b', data=data)
# plt.scatter('a', 'c', data=data)
# plt.xlabel('entry a')
# plt.ylabel('entry b')
# plt.show()

### Data

In [3]:
# Make global decision as to which data to initialise network on. Current options are "toy" or "USA"
#init = "toy"
# behav = False
shock = False

#### Toy Model
Toy model constructed on 5 fake occupations with pre-determined employment, unemployment, vacancies, target demand, and wages.

In [4]:
T = 600
delta_u = 0.01
delta_v = 0.005
gamma_u = gamma_v = gamma = 0.01
# Import information about relevant files to employment/unemployment, target demand, vacancies, etc.

A = pd.read_csv(path+"data/small_adj_full.csv", delimiter=';', decimal=',', header=None)
employment = pd.read_csv(path+"data/employed.csv", header = None)
unemployment = pd.read_csv(path+"data/unemployed.csv", header = None)
vacancies = pd.read_csv(path+"data/vacancies.csv", header = None)
demand_target = employment + vacancies
wages = pd.DataFrame(np.round(np.random.normal(50000, 10000, 5)), columns = ['Wages'])
mod_data =  {"A": A, "employment": employment, 
             'unemployment':unemployment, 'vacancies':vacancies, 
             'demand_target': demand_target, 'wages': wages}



#### US Model
Model constructed using 464 occupations from US Bureau of Labor Statistics Data and IPUMS.
Data input from replicaiton code in dRC et al 2021: https://zenodo.org/records/4453162


In [5]:
# ###################################
# # INITIAL MODEL CONDITIONS ########
# ###################################
# T = 45
# delta_u = 0.015
# delta_v = 0.01
# gamma_u = gamma_v = gamma = 0.06

# A = pd.read_csv(path+"dRC_Replication/data/occupational_mobility_network.csv", header=None)
# employment = round(pd.read_csv(path+"dRC_Replication/data/ipums_employment_2016.csv", header = 0).iloc[:, [4]]/1000)
# # Crude approximation using avg unemployment rate of ~5% - should aim for occupation-specific unemployment rates
# unemployment = round(employment*(0.05/0.95))
# # Less crude approximation using avg vacancy rate - should still aim for occupation-specific vacancy rates
# vac_rate_base = pd.read_csv(path+"dRC_Replication/data/vacancy_rateDec2000.csv").iloc[:, 2].mean()/100
# vacancies = round(employment*vac_rate_base/(1-vac_rate_base))
# # Needs input data...
# demand_target = employment + vacancies
# wages = pd.read_csv(path+"dRC_Replication/data/ipums_variables.csv")[['median_earnings']]
# mod_data =  {"A": A, "employment": employment, 
#              'unemployment':unemployment, 'vacancies':vacancies, 
#              'demand_target': demand_target, 'wages': wages}


### Initialise Network

In [6]:
#%%file init.py
### Function and condition to initialise network

def initialise(n_occ, employment, unemployment, vacancies, demand_target, A, wages):
    """ Makes a list of occupations with initial conditions
       Args:
           n_occ: number of occupations initialised
           employment: vector with employment of each occupation
           unemployment: vector with unemployment of each occupation
           vacancies: vector with vacancies of each occupation
           demand_target: vector with (initial) target_demand for each occupation (never updated)
           A: adjacency matrix of network (not including auto-transition probability)
           wages: vector of wages of each occupation

       Returns:
            occupations: list of occupations with above attributes
            vacancies: list of vacancies with occupation id, wage, and list of applicants
       """
    occs = []
    vac_list = []
    ids = 0
    for i in range(0, n_occ):
        # appending relevant number of vacancies to economy-wide vacancy list
        for v in range(round(vacancies.iat[i,0])):
            vac_list.append(vac(i, [], wages.iat[i,0]))
            
        occ = occupation(i, [], A[i] > 0, A[i],
                         (employment.iat[i,0] + vacancies.iat[i,0]), 
                         demand_target.iat[i,0], [], wages.iat[i,0])
        # creating the workers of occupation i and attaching to occupation
        ## adding employed workers
        for e in range(round(employment.iat[i,0])):
            # Assume they have all at least 1 t.s. of employment
            occ.list_of_workers.append(worker(occ.occupation_id, True, False, 1, 0, wages.iat[i,0], False, [occ.occupation_id], random.randint(0, 9)))
            ## adding unemployed workers
            # Could consider adding random initial unemployment durations...for now no one becomes longterm unemployed until 6 time steps in
        for u in range(round(unemployment.iat[i,0])):
            # Assigns time unemployed from absolute value of normal distribution....
            occ.list_of_workers.append(worker(occ.occupation_id, False, False, 0, abs(int(np.random.normal(0, 2))), wages.iat[i,0], False,
                                                     [occ.occupation_id], 
                                                      random.randint(0, 9)))
        occs.append(occ)
        ids += 1
    return occs, vac_list
    

In [7]:
####################
# Testing Cell #####
####################

## Model Run
### Function

In [8]:
####################
# Model Run ########
####################
def run_sim(behav_spec, data, time_steps, runs, d_u, d_v):
    import numpy as np
    import pandas as pd
    # Records variables of interest
    record = pd.DataFrame(columns=['Sim', 'Time', 'Occupation_ID', 'Workers', 'Employment', 'Unemployment', 'Vacancies', 'LT Unemployed Persons', 'Target_Demand'])
    print(record)
    for run in range(runs):
        #print("Running ", init, " model.")
        #print("RUN: ", run)
        # Initialise occupational mobility network
        net_temp, vacs = initialise(len(data['A']), data['employment'], data['unemployment'], data['vacancies'], data['demand_target'], data['A'], data['wages'])
        for t in range(time_steps):
            #print("TIME: ", t)
            if t == 400 and shock:
                print("initiatied shock!")
                net_temp[0].target_demand += 25
                net_temp[1].target_demand += 50
                net_temp[2].target_demand += 50
                net_temp[3].target_demand += 50
                net_temp[4].target_demand = 100

            # Ensure number of workers in economy has not changed
            assert(sum(map(lambda x: len(x.list_of_workers), net_temp)) == employment.sum().item() + unemployment.sum().item())
            for occ in net_temp:

                ### SEPARATIONS
                occ.separate_workers(d_u)

                # Ensure that separated workers have been reassigned appropriately 
                # (ie. that people move witihin the same occupation from employed to unemployed 
                # and that the total number of workers iwthin an occupation is (at this stage) 
                # the same as before separations
                if t > 0:
                    temp = record.loc[(record['Sim'] == run) & (record['Occupation_ID'] == occ.occupation_id) & (record['Time'] == t-1)]
                    assert(temp.Employment.item() - sum(wrkr.employed for wrkr in occ.list_of_workers) ==
                           sum(not(wrkr.employed) for wrkr in occ.list_of_workers) - temp.Unemployment.item())
                    assert(len(occ.list_of_workers) == temp.Workers.item())

                ### APPLICATIONS
                # Questions to verify:
                # - CANNOT be fired and apply in same time step ie. time_unemployed > 0
                # - CAN be rejected and apply in the same time step - no protected attribute
                unemp = [el for el in occ.list_of_workers if not(el.employed) and el.time_unemployed > 0]
                for u in unemp:
                    u.search_and_apply(net_temp, vacs, behav_spec)

            ### HIRING
            # Ordering of hiring randomised to ensure list order does not matter in filling vacancies...
            # ....might be better to do this using an unordered set?
            for v_open in sorted(vacs,key=lambda _: random.random()):
                # Removes any applicants that have already been hired in another vacancy
                v_open.applicants[:] = [app for app in v_open.applicants if not(app.hired)]
                if len([app for app in v_open.applicants if not(app.hired)]) > 0:
                    v_open.hire(net_temp)
                    vacs.remove(v_open)
                    assert(len(v_open.applicants) == 0)
                else:
                    pass

            ### OPEN VACANCIES
            # Update vacancies after all shifts have taken place
            # Could consider making this a function of the class itself
            for occ in net_temp:
                # Update all workers
                occ.update_workers()
                emp = sum(wrkr.employed for wrkr in occ.list_of_workers)
                occ.current_demand = bus_cycle_demand(len([v_open for v_open in vacs if v_open.occupation_id == occ.occupation_id]) + emp, t, 0.005, 27)
                vac_prob = d_v + ((1 - d_v) * (gamma * max(0, occ.target_demand - occ.current_demand))) / (emp + 1)
                print("vac prob:", vac_prob)
                for v in range(int(np.random.binomial(emp, vac_prob))):
                    vacs.append(vac(occ.occupation_id, [], occ.wage))

                ### UPDATE INDICATOR RECORD
                # Record of indicators of interest (simulation number, occ, # workers, employed, unemployed, vacancies, long_term_unemployed)
                record.loc[len(record)]= [run, 
                                          t,
                                          occ.occupation_id,
                                          len(occ.list_of_workers),
                                          sum(wrkr.employed for wrkr in occ.list_of_workers),
                                          sum(not(wrkr.employed) for wrkr in occ.list_of_workers),
                                          len([v_open for v_open in vacs if v_open.occupation_id == occ.occupation_id]),
                                          sum(wrkr.longterm_unemp for wrkr in occ.list_of_workers),
                                          occ.target_demand]

        print("Done after ", t + 1, " time steps.")
    print("Done after ", run + 1, " runs.")
    return(record)


In [9]:
#test_params
# list of parameters to explore
δ_u_list = [0.0150001 + i * 0.00025 for i in range(1,9)][0:2]
δ_v_list = [0.009001 + i * 0.00025 for i in range(1,18)][0:2]
amplitude_list = [0.06 + 0.0025*i for i in range(9)][0:2]
τ_list = [i for i in range(2,8)][0:2]
amplitude_list


[0.06, 0.0625]

In [10]:
T = 600
#del_u = 0.01
#del_v = 0.005
gamma_u = gamma_v = gamma = 0.015

In [11]:
# def run_sim(behav_spec, data, time_steps, runs, d_u, d_v):
#sim_record_f_all = run_sim(False, mod_data, T, 3, del_u, del_v)
#sim_record_t_all = run_sim(False, mod_data, T, 3, del_u, del_v)

## Results

In [12]:
# Results specifications (whether to save as final or not)
final = False
fullpdf = True

In [13]:
# ue_vac_f = record1_f.loc[:,['Time', 'Workers', 'Unemployment', 'Vacancies', 'Target_Demand']].groupby(['Time']).sum().reset_index()
# ue_vac_f['UE Rate'] = ue_vac_f['Unemployment'] / ue_vac_f['Workers']
# ue_vac_f['Vac Rate'] = ue_vac_f['Vacancies'] / ue_vac_f['Target_Demand']
# ue_vac_f = ue_vac_f.loc[(ue_vac_f.Time > 200) & (ue_vac_f.Time < 227)]

# ue_vac_t = record1_t.loc[:,['Time', 'Workers', 'Unemployment', 'Vacancies', 'Target_Demand']].groupby(['Time']).sum().reset_index()
# ue_vac_t['UE Rate'] = ue_vac_t['Unemployment'] / ue_vac_t['Workers']
# ue_vac_t['Vac Rate'] = ue_vac_t['Vacancies'] / ue_vac_t['Target_Demand']
# ue_vac_t = ue_vac_t.loc[(ue_vac_t.Time > 200) & (ue_vac_t.Time < 227)]


# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))

# ax1.plot(ue_vac_f['UE Rate'], ue_vac_f['Vac Rate'])
# ax1.scatter(ue_vac_f['UE Rate'], ue_vac_f['Vac Rate'], c=ue_vac_f['Time'], s=100, lw=0)
# ax1.set_title("Non-behavioural")
# ax1.set_xlabel("UE Rate")
# ax1.set_ylabel("Vacancy Rate")

# ax2.plot(ue_vac_t['UE Rate'], ue_vac_t['Vac Rate'])
# ax2.set_title("Behavioural")
# ax2.scatter(ue_vac_t['UE Rate'], ue_vac_t['Vac Rate'], c=ue_vac_t['Time'], s=100, lw=0)
# ax2.set_xlabel("UE Rate")
# ax2.set_ylabel("Vacancy Rate")

# fig.text(0.05,0.95,txt, transform=fig.transFigure, size=18)
# fig.suptitle("Beveridge Curve", fontweight = 'bold')
# fig.tight_layout()


In [14]:
with PdfPages('multipage_pdf.pdf') as pdf:
    for del_u in [0.02]:#δ_u_list:
        for del_v in [0.01, 0.015, 0.018, 0.02, 0.022, 0.025, 0.03]: #δ_v_list:
                print(["δ_u: " + str(del_u) + "; δ_v: " + str(del_v)])
                sim_record_f_all = run_sim(False, mod_data, T, 1, del_u, del_v)
                sim_record_t_all = run_sim(True, mod_data, T, 1, del_u, del_v)
                sim_record_t = sim_record_t_all.loc[(sim_record_t_all.Time > 0)]
                sim_record_f = sim_record_f_all.loc[(sim_record_f_all.Time > 0)]
                record1_f = sim_record_f.loc[(sim_record_f.Sim == 0)]
                record1_t = sim_record_t.loc[(sim_record_t.Sim == 0)]

                txt = ["δ_u: " + str(del_u) + "; δ_v: " + str(del_v)]
                ue_vac_f = record1_f.loc[:,['Time', 'Workers', 'Unemployment', 'Vacancies', 'Target_Demand']].groupby(['Time']).sum().reset_index()
                ue_vac_f['UE Rate'] = ue_vac_f['Unemployment'] / ue_vac_f['Workers']
                ue_vac_f['Vac Rate'] = ue_vac_f['Vacancies'] / ue_vac_f['Target_Demand']
                ue_vac_f = ue_vac_f.loc[(ue_vac_f.Time > 200) & (ue_vac_f.Time < 227)]

                ue_vac_t = record1_t.loc[:,['Time', 'Workers', 'Unemployment', 'Vacancies', 'Target_Demand']].groupby(['Time']).sum().reset_index()
                ue_vac_t['UE Rate'] = ue_vac_t['Unemployment'] / ue_vac_t['Workers']
                ue_vac_t['Vac Rate'] = ue_vac_t['Vacancies'] / ue_vac_t['Target_Demand']
                ue_vac_t = ue_vac_t.loc[(ue_vac_t.Time > 200) & (ue_vac_t.Time < 227)]


                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))

                ax1.plot(ue_vac_f['UE Rate'], ue_vac_f['Vac Rate'])
                ax1.scatter(ue_vac_f['UE Rate'], ue_vac_f['Vac Rate'], c=ue_vac_f['Time'], s=100, lw=0)
                ax1.set_title("Non-behavioural")
                ax1.set_xlabel("UE Rate")
                ax1.set_ylabel("Vacancy Rate")

                ax2.plot(ue_vac_t['UE Rate'], ue_vac_t['Vac Rate'])
                ax2.set_title("Behavioural")
                ax2.scatter(ue_vac_t['UE Rate'], ue_vac_t['Vac Rate'], c=ue_vac_t['Time'], s=100, lw=0)
                ax2.set_xlabel("UE Rate")
                ax2.set_ylabel("Vacancy Rate")

                fig.text(0.05,0.95,txt, transform=fig.transFigure, size=18)
                fig.suptitle("Beveridge Curve", fontweight = 'bold')
                fig.tight_layout()


                if final:
                    plt.savefig('../output/overall_economy_comparison_behav.jpg', dpi = 300)
                elif fullpdf:
                    pdf.savefig()
                    plt.close()
                else:
                    plt.show()


                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))

                totals = record1_f.loc[:,['Time', 'Workers','Employment', 'Unemployment', 'Vacancies', 'LT Unemployed Persons', 'Target_Demand']].groupby(['Time']).sum()
                lgd = []
                for column in totals[1:]:
                    ax1.plot(totals[column])
                    lgd.append(column)
                ax1.set_title("Non-behavioural")

                totals = record1_t.loc[:,['Time', 'Workers','Employment', 'Unemployment', 'Vacancies', 'LT Unemployed Persons', 'Target_Demand']].groupby(['Time']).sum()
                for column in totals[1:]:
                    ax2.plot(totals[column])
                ax2.set_title('Behavioural')

                fig.suptitle("Overall Economy Performance: Indicators over Time", fontweight = 'bold')
                plt.legend(list(lgd), loc="center", ncol=1, fontsize = 8)
                fig.tight_layout()

                if final:
                    plt.savefig('../output/overall_economy_comparison_behav.jpg', dpi = 300)
                elif fullpdf:
                    pdf.savefig()
                    plt.close()
                else:
                    plt.show()

            # # LONG-TERM UNEMPLOYMENT RATE AND LEVELS
            # # LT Unemployed per occupation

            # ltue = record1_t.loc[:,['Time', 'Occupation_ID','Workers', 'LT Unemployed Persons']].groupby(['Time', 'Occupation_ID']).sum().reset_index()
            # ltue['LTUE Rate'] = ltue['LT Unemployed Persons'] / ltue['Workers']
            # lgd = []
            # for g in np.unique(ltue.Occupation_ID):
            #     temp = ltue[(ltue['Occupation_ID'] == g)][['LTUE Rate', 'Time']]
            #     plt.plot(temp['Time'], temp['LTUE Rate'])
            #     lgd.append(int(g))
            # plt.title("LT Unemployment Rate per Occ", fontweight = 'bold')
            # plt.legend(list(lgd), loc="center left", ncol=1, title = "Occ")
            # if final:
            #     plt.savefig('../output/ltuer_occ_base.jpg', dpi = 300)
            # else:
            #     plt.show()
            # plt.close(fig)

            # fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))

            # ltue = record1_f.loc[:,['Time', 'Occupation_ID','Workers', 'LT Unemployed Persons']].groupby(['Time', 'Occupation_ID']).sum().reset_index()
            # ltue['LTUE Rate'] = ltue['LT Unemployed Persons'] / ltue['Workers']
            # lgd = []
            # for g in np.unique(ltue.Occupation_ID):
            #     temp = ltue[(ltue['Occupation_ID'] == g)][['LTUE Rate', 'Time']]
            #     ax1.plot(temp['Time'], temp['LTUE Rate'])
            # ax1.set_title("Non-behavioural")

            # ltue = record1_t.loc[:,['Time', 'Occupation_ID','Workers', 'LT Unemployed Persons']].groupby(['Time', 'Occupation_ID']).sum().reset_index()
            # ltue['LTUE Rate'] = ltue['LT Unemployed Persons'] / ltue['Workers']
            # lgd = []
            # for g in np.unique(ltue.Occupation_ID):
            #     temp = ltue[(ltue['Occupation_ID'] == g)][['LTUE Rate', 'Time']]
            #     ax2.plot(temp['Time'], temp['LTUE Rate'])
            #     lgd.append(int(g))
            # ax2.set_title('Behavioural')
            # fig.suptitle("LT Unemployment Rate per Occ", fontweight = 'bold')
            # fig.tight_layout()


            # if final:
            #     plt.savefig('../output/ltuer_occ_comparison_behav.jpg', dpi = 300)
            # else:
            #     plt.show()
            # plt.close()

            # fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))
            # colors = ['b','g','r','y','m']

            # sim_record_f['LTUE Rate'] = sim_record_f['LT Unemployed Persons'] / sim_record_f['Workers']
            # for o in np.unique(sim_record_f.Occupation_ID):
            #     temp = sim_record_f[(sim_record_f['Occupation_ID'] == o)][['LTUE Rate', 'Time', 'Sim']]
            #     temp_mean = temp.loc[:, ['Time', 'LTUE Rate']].groupby(['Time']).mean().reset_index()
            #     spl = splrep(temp_mean['Time'], temp_mean['LTUE Rate'], s=2)
            #     for g in np.unique(temp.Sim):
            #         ax1.plot(temp[(temp['Sim'] == g)]['Time'], temp[(temp['Sim'] == g)]['LTUE Rate'], alpha = 0.05, color = colors[o])
            #     ax1.plot(temp_mean['Time'], BSpline(*spl)(temp_mean['Time']), '-', color = colors[o])
            # ax1.set_title("Non-behavioural")
            # fig.suptitle("LT Unemployment Rate", fontweight = 'bold')

            # sim_record_t['LTUE Rate'] = sim_record_t['LT Unemployed Persons'] / sim_record_t['Workers']
            # for o in np.unique(sim_record_t.Occupation_ID):
            #     temp = sim_record_t[(sim_record_t['Occupation_ID'] == o)][['LTUE Rate', 'Time', 'Sim']]
            #     temp_mean = temp.loc[:, ['Time', 'LTUE Rate']].groupby(['Time']).mean().reset_index()
            #     spl = splrep(temp_mean['Time'], temp_mean['LTUE Rate'], s=3)
            #     for g in np.unique(temp.Sim):
            #         ax2.plot(temp[(temp['Sim'] == g)]['Time'], temp[(temp['Sim'] == g)]['LTUE Rate'], alpha = 0.05, color = colors[o])
            #     ax2.plot(temp_mean['Time'], BSpline(*spl)(temp_mean['Time']), '-', color = colors[o])
            # ax2.set_title("Behavioural")
            # fig.suptitle("LT Unemployment Rate by Occupation", fontweight = 'bold')

            # fig.tight_layout()

            # if final:
            #     plt.savefig('../output/ltuer_sim_comparison_behav.jpg', dpi = 300)
            # else:
            #     plt.show()
            # plt.close()

                # LONGTERUM UNEMPLOYMENT RATE BY OCCUPATION
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))
                colors = colors = ['b','g','r','y','m']

                sim_record_f['UE Rate'] = sim_record_f['Unemployment'] / sim_record_f['Workers']
                for o in np.unique(sim_record_f.Occupation_ID):
                    temp = sim_record_f[(sim_record_f['Occupation_ID'] == o)][['UE Rate', 'Time', 'Sim']]
                    temp_mean = temp.loc[:, ['Time', 'UE Rate']].groupby(['Time']).mean().reset_index()
                    spl = splrep(temp_mean['Time'], temp_mean['UE Rate'], s=2)
                    for g in np.unique(temp.Sim):
                        ax1.plot(temp[(temp['Sim'] == g)]['Time'], temp[(temp['Sim'] == g)]['UE Rate'], alpha = 0.05, color = colors[o])
                    ax1.plot(temp_mean['Time'], BSpline(*spl)(temp_mean['Time']), '-', color = colors[o])
                ax1.set_title("Non-behavioural")
                fig.suptitle("Unemployment Rate", fontweight = 'bold')

                sim_record_t['UE Rate'] = sim_record_t['Unemployment'] / sim_record_t['Workers']
                for o in np.unique(sim_record_t.Occupation_ID):
                    temp = sim_record_t[(sim_record_t['Occupation_ID'] == o)][['UE Rate', 'Time', 'Sim']]
                    temp_mean = temp.loc[:, ['Time', 'UE Rate']].groupby(['Time']).mean().reset_index()
                    spl = splrep(temp_mean['Time'], temp_mean['UE Rate'], s=3)
                    for g in np.unique(temp.Sim):
                        ax2.plot(temp[(temp['Sim'] == g)]['Time'], temp[(temp['Sim'] == g)]['UE Rate'], alpha = 0.05, color = colors[o])
                    ax2.plot(temp_mean['Time'], BSpline(*spl)(temp_mean['Time']), '-', color = colors[o])
                ax2.set_title("Behavioural")
                fig.suptitle("Unemployment Rate by Occupation", fontweight = 'bold')
                fig.tight_layout()

                if final:
                    plt.savefig('../output/occ_ltue.jpg', dpi = 300)
                elif fullpdf:
                    pdf.savefig()
                    plt.close()
                else:
                    plt.show()

                #### Occupations: Employment, Unemployment, Worker, Vacancy, and Longterm Unemployed Levels
                # # Overall indicators per occupation
                # ids = np.unique(record1_t.Occupation_ID)
                # fig = plt.figure(constrained_layout = False)

                # occ_totals = record1_t.loc[:,['Time', 'Occupation_ID', 'Workers','Employment', 'Unemployment', 'Vacancies', 'LT Unemployed Persons', 'Target_Demand']]
                # for occ in ids:
                #     gtmp = occ_totals[(occ_totals['Occupation_ID'] == occ)].loc[:, ['Time', 'Workers', 'Employment', 'Unemployment', 'Vacancies', 'LT Unemployed Persons', 'Target_Demand']].groupby(['Time']).sum()
                #     # Indicators (workers, employment, etc)
                #     fig.add_subplot(3, 2, int(occ)+1, title = f'Occ: {int(occ)}')
                #     lgd = []
                #     for column in gtmp[1:]:
                #         plt.plot(gtmp[column])
                #         lgd.append(column)

                # fig.suptitle("Economy Performance per Economy: Indicators over Time", fontweight = 'bold')
                # fig.legend(list(lgd), bbox_to_anchor=(0.9, 0.3), ncols = 1, title_fontsize = "6", fontsize="8")
                # fig.subplots_adjust(wspace=0.2, hspace = 0.75)
                # if final:
                #     plt.savefig('../output/occ_perf_base.jpg', dpi = 300)
                # else:
                #     plt.show()
                # plt.close(fig)
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))

                ltue_tot = sim_record_f.loc[:,['Sim','Time', 'Workers', 'Unemployment', 'LT Unemployed Persons']].groupby(['Time', 'Sim']).sum().reset_index()
                ltue_tot['UE Rate'] = ltue_tot['Unemployment'] / ltue_tot['Workers']
                ltue_tot['LTUE Rate'] = ltue_tot['LT Unemployed Persons'] / ltue_tot['Workers']
                ltue_mean = ltue_tot.loc[:, ['Time', 'LTUE Rate']].groupby(['Time']).mean().reset_index()
                spl = splrep(ltue_mean['Time'], ltue_mean['LTUE Rate'], s=2)
                for g in np.unique(ltue_tot.Sim):
                    temp = ltue_tot[(ltue_tot['Sim'] == g)][['LTUE Rate', 'Time']]
                    ax1.plot(temp['Time'], temp['LTUE Rate'], alpha = 0.3)
                    #temp = ltue_tot[(ltue_tot['Sim'] == g)][['UE Rate', 'Time']]
                    #ax1.plot(temp['Time'], temp['UE Rate'])
                ax1.plot(ltue_mean['Time'], BSpline(*spl)(ltue_mean['Time']), '-', label='s=0',  color = 'black')
                ax1.set_title("Non-behavioural")
                fig.suptitle("LT Unemployment Rate", fontweight = 'bold')


                ltue_tot = sim_record_t.loc[:,['Sim','Time', 'Workers', 'Unemployment','LT Unemployed Persons']].groupby(['Time', 'Sim']).sum().reset_index()
                ltue_tot['UE Rate'] = ltue_tot['Unemployment'] / ltue_tot['Workers']
                ltue_tot['LTUE Rate'] = ltue_tot['LT Unemployed Persons'] / ltue_tot['Workers']
                ltue_mean = ltue_tot.loc[:, ['Time', 'LTUE Rate']].groupby(['Time']).mean().reset_index()
                spl = splrep(ltue_mean['Time'], ltue_mean['LTUE Rate'], s=2)
                for g in np.unique(ltue_tot.Sim):
                    temp = ltue_tot[(ltue_tot['Sim'] == g)][['LTUE Rate', 'Time']]
                    ax2.plot(temp['Time'], temp['LTUE Rate'], alpha = 0.2)
                    #temp = ltue_tot[(ltue_tot['Sim'] == g)][['UE Rate', 'Time']]
                    #ax2.plot(temp['Time'], temp['UE Rate'])
                ax2.plot(ltue_mean['Time'], BSpline(*spl)(ltue_mean['Time']), '-', label='s=0',  color = 'black')
                ax2.set_title("Behavioural")
                fig.suptitle("LT Unemployment Rate", fontweight = 'bold')
                fig.tight_layout()

                if final:
                    plt.savefig('../output/ltuer_sim_comparison_behav.jpg', dpi = 300)
                elif fullpdf:
                    pdf.savefig()
                    plt.close()
                else:
                    plt.show()


                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))

                ltue_tot = sim_record_f.loc[:,['Sim','Time', 'Workers', 'Unemployment', 'LT Unemployed Persons']].groupby(['Time', 'Sim']).sum().reset_index()
                ltue_tot['UE Rate'] = ltue_tot['Unemployment'] / ltue_tot['Workers']
                ltue_tot['LTUE Rate'] = ltue_tot['LT Unemployed Persons'] / ltue_tot['Workers']
                ltue_mean = ltue_tot.loc[:, ['Time', 'UE Rate']].groupby(['Time']).mean().reset_index()
                spl = splrep(ltue_mean['Time'], ltue_mean['UE Rate'], s=2)
                for g in np.unique(ltue_tot.Sim):
                    temp = ltue_tot[(ltue_tot['Sim'] == g)][['UE Rate', 'Time']]
                    ax1.plot(temp['Time'], temp['UE Rate'], alpha = 0.3)
                    #temp = ltue_tot[(ltue_tot['Sim'] == g)][['UE Rate', 'Time']]
                    #ax1.plot(temp['Time'], temp['UE Rate'])
                ax1.plot(ltue_mean['Time'], BSpline(*spl)(ltue_mean['Time']), '-', label='s=0',  color = 'black')
                ax1.set_title("Non-behavioural")
                fig.suptitle("Unemployment Rate", fontweight = 'bold')


                ltue_tot = sim_record_t.loc[:,['Sim','Time', 'Workers', 'Unemployment','LT Unemployed Persons']].groupby(['Time', 'Sim']).sum().reset_index()
                ltue_tot['UE Rate'] = ltue_tot['Unemployment'] / ltue_tot['Workers']
                ltue_tot['LTUE Rate'] = ltue_tot['LT Unemployed Persons'] / ltue_tot['Workers']
                ltue_mean = ltue_tot.loc[:, ['Time', 'UE Rate']].groupby(['Time']).mean().reset_index()
                spl = splrep(ltue_mean['Time'], ltue_mean['UE Rate'], s=2)
                for g in np.unique(ltue_tot.Sim):
                    temp = ltue_tot[(ltue_tot['Sim'] == g)][['UE Rate', 'Time']]
                    ax2.plot(temp['Time'], temp['UE Rate'], alpha = 0.2)
                    #temp = ltue_tot[(ltue_tot['Sim'] == g)][['UE Rate', 'Time']]
                    #ax2.plot(temp['Time'], temp['UE Rate'])
                ax2.plot(ltue_mean['Time'], BSpline(*spl)(ltue_mean['Time']), '-', label='s=0',  color = 'black')
                ax2.set_title("Behavioural")
                fig.suptitle("Unemployment Rate", fontweight = 'bold')
                fig.tight_layout()

                if final:
                    plt.savefig('../output/ltuer_sim_comparison_behav.jpg', dpi = 300)
                elif fullpdf:
                    pdf.savefig()
                    plt.close()
                else:
                    plt.show()

['δ_u: 0.02; δ_v: 0.01']
Empty DataFrame
Columns: [Sim, Time, Occupation_ID, Workers, Employment, Unemployment, Vacancies, LT Unemployed Persons, Target_Demand]
Index: []
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.010288349514563106
vac prob: 0.010147761194029851
vac prob: 0.010090548780487806
vac prob: 0.01025603448275862
vac prob: 0.01024264705882353
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.010154932698898281
vac prob: 0.010394287541525509
vac prob: 0.01032218133084025
vac prob: 0.010135948133062431
vac prob: 0.010403893817167723
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.010448600284057375
vac prob: 0.010730111325361757
vac prob: 0.010250949526123937
vac prob: 0.01026236253175169
vac prob: 0.01047072771516724
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.010612606883794626
vac prob: 0.010590

vac prob: 0.015242553788101233
vac prob: 0.01711055288563955
vac prob: 0.01496077686741109
vac prob: 0.017097440199809015
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.015551821353738313
vac prob: 0.01539193056781196
vac prob: 0.017725744955365476
vac prob: 0.015518032437310127
vac prob: 0.017407953214581315
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.01556875
vac prob: 0.015408940397350994
vac prob: 0.01756833976833977
vac prob: 0.015535
vac prob: 0.017933561643835617
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.016060528772258408
vac prob: 0.015425950226890025
vac prob: 0.018026701483408916
vac prob: 0.015738984483187925
vac prob: 0.018702891515061104
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.0165739980612831
vac prob: 0.01530876200137245
vac prob: 0.01831605910271901
vac prob: 0.0163372542510248

vac prob: 0.02059091260657849
vac prob: 0.021096911463803356
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.01648770688202852
vac prob: 0.017104294719292243
vac prob: 0.019289342538333563
vac prob: 0.020870751213927993
vac prob: 0.02121983547828015
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.01754712639359689
vac prob: 0.017253554877437346
vac prob: 0.01907914514429062
vac prob: 0.020001412084727128
vac prob: 0.021209594484901104
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.017263080769096866
vac prob: 0.017408585561347726
vac prob: 0.018875677191502745
vac prob: 0.020274612320522814
vac prob: 0.020932087572603265
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.01782175061140977
vac prob: 0.017569803004400943
vac prob: 0.018583665596498364
vac prob: 0.020557967941756898
vac prob: 0.02119982391443554
sep_p

sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.019180147015985676
vac prob: 0.018223324066069718
vac prob: 0.01951716749040735
vac prob: 0.018580172788414687
vac prob: 0.019444126237915228
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.01950487945668743
vac prob: 0.01912276403402702
vac prob: 0.019723178708259022
vac prob: 0.019078123057552156
vac prob: 0.019328114118042694
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.01886295794864088
vac prob: 0.01911979376252935
vac prob: 0.019824007286029725
vac prob: 0.018578202276275997
vac prob: 0.019442144649826854
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.0185491815760202
vac prob: 0.019301197643032328
vac prob: 0.019508384803025008
vac prob: 0.01833064978431109
vac prob: 0.01955350091117621
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac p

sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.020836164325139732
vac prob: 0.018698529972799527
vac prob: 0.01784811102829168
vac prob: 0.01877062719441944
vac prob: 0.018702891515061104
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.021607097995907383
vac prob: 0.018534977359441866
vac prob: 0.01751531055705631
vac prob: 0.01878665677766536
vac prob: 0.01776054812040079
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.021621289818548356
vac prob: 0.018026475286138485
vac prob: 0.0179675398153917
vac prob: 0.01831424525857573
vac prob: 0.018622855022366554
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.021249998840582904
vac prob: 0.017869116479306047
vac prob: 0.018250392735764197
vac prob: 0.019063711289114594
vac prob: 0.019082956892769898
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac 

...

sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.018139423365878458
vac prob: 0.021248834789040768
vac prob: 0.01860995612516304
vac prob: 0.020876959536992493
vac prob: 0.022822538169869767
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.01845282371722245
vac prob: 0.02082367937800566
vac prob: 0.018913329625374745
vac prob: 0.02031199490063562
vac prob: 0.023148324544797812
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.018468813436901304
vac prob: 0.02105759815488892
vac prob: 0.019326073118382383
vac prob: 0.020614321353738314
vac prob: 0.022699822203798493
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.018485714285714286
vac prob: 0.021296153846153847
vac prob: 0.018753048780487804
vac prob: 0.021223837209302324
vac prob: 0.022120810810810812
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.018502615134527268
va

vac prob: 0.021644349387152817
vac prob: 0.023810185574361487
vac prob: 0.017049246771209392
vac prob: 0.01764391941407653
vac prob: 0.02341055936128214
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.021637621468786422
vac prob: 0.022285414754494436
vac prob: 0.01729143431632137
vac prob: 0.018571445069707756
vac prob: 0.024223230860729653
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.020872521671559844
vac prob: 0.022037744269264987
vac prob: 0.01745035229176463
vac prob: 0.018320461023887964
vac prob: 0.02387955917396216
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.020859503883884304
vac prob: 0.022262121386386165
vac prob: 0.017268022330436884
vac prob: 0.018071507153441944
vac prob: 0.023222032496444105
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.02084431792427562
vac prob: 0.022009382758838056
vac 

vac prob: 0.018203442161371826
vac prob: 0.020873702024802605
vac prob: 0.018117055547657954
vac prob: 0.015738984483187925
vac prob: 0.020624178722135268
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.018518604854206117
vac prob: 0.020675545644637135
vac prob: 0.018133191626327
vac prob: 0.0165387202597096
vac prob: 0.020383155039437308
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.018532821434824175
vac prob: 0.020689829255185734
vac prob: 0.018057184747814425
vac prob: 0.015959785063537568
vac prob: 0.020397485934665595
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.018544498456840475
vac prob: 0.020283659633561553
vac prob: 0.017979324087661817
vac prob: 0.015971506187367042
vac prob: 0.020537174105706418
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.018253841936372577
vac prob: 0.02029220887752655
vac 

sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.020859503883884304
vac prob: 0.020061201033153286
vac prob: 0.019689995973935587
vac prob: 0.015952923254199336
vac prob: 0.01872645764048203
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.021216331300052918
vac prob: 0.020045920239579802
vac prob: 0.01897180731123356
vac prob: 0.015560164132045185
vac prob: 0.019048957699012065
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.02119979932062925
vac prob: 0.02002928152571222
vac prob: 0.018857972470715127
vac prob: 0.016719594707538473
vac prob: 0.019737887781145356
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
vac prob: 0.021960165472751182
vac prob: 0.020423269521778052
vac prob: 0.01874446392185333
vac prob: 0.017125288139350454
vac prob: 0.019842412872902802
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
sep_prob:  0.02
v

KeyboardInterrupt: 

#### Overall Economy: Employment, Unemployment, Worker, Vacancy, and Longterm Employed Levels

In [ ]:
# # Indicators in overall economy
# totals = record1_f.loc[:,['Time', 'Workers','Employment', 'Unemployment', 'Vacancies', 'LT Unemployed Persons', 'Target_Demand']].groupby(['Time']).sum()
# lgd = []
# for column in totals[1:]:
#     plt.plot(totals[column])
#     lgd.append(column)
#     plt.legend(list(lgd), loc="center", ncol=1)
# plt.title("Overall Economy Performance: Indicators over Time", fontweight = 'bold')
# if final:
#     plt.savefig('../output/overall_economy_base.jpg', dpi = 300)
# else:
#     plt.show()
# plt.close()

In [ ]:
del_v
#(1.77 - 0.005)/(0.01)

#1.77 = 0.005 + 0.01*x
#0.005 + 0.01 * max(0, occ.current_demand - occ.target_demand)/(sum(wrkr.employed for wrkr in occ.list_of_workers) + 1)